In [ ]:
import tensorflow as tf
import os
import numpy as np
from PIL import Image
import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE

tfk = tf.keras
tfkl = tf.keras.layers

In [ ]:
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
#!unzip dataset.zip

In [ ]:
# Dataset folders 
dataset_dir = '../input/dataset21/dataset1/training_set1'
validation_dir = '../input/dataset21/dataset1/validation_set1'

input_shape = (256, 256, 3)
epochs = 50

In [ ]:
# Plot example images from dataset
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

num_row = len(labels)//2
num_col = len(labels)//num_row
fig, axes = plt.subplots(num_row, num_col, figsize=(2*num_row,15*num_col))
for i in range(len(labels)):
  if i < len(labels):
    class_imgs = next(os.walk('{}/{}/'.format(dataset_dir, labels[i])))[2]
    class_img = class_imgs[0]
    img = Image.open('{}/{}/{}'.format(dataset_dir, labels[i], class_img))
    ax = axes[i//num_col, i%num_col]
    ax.imshow(np.array(img))
    ax.set_title('{}'.format(labels[i]))
plt.tight_layout()
plt.show()

In [ ]:
#Augmentation on training

# Create an instance of ImageDataGenerator with Data Augmentation and data rescale
aug_train_data_gen = ImageDataGenerator(rotation_range=30,
                         height_shift_range=50,
                         width_shift_range=50,
                         zoom_range=0.3,
                         #brightness_range=[0.01, 0.1],
                         horizontal_flip=True,
                         vertical_flip=True,
                         fill_mode='reflect',
                         rescale=1/255.)
                         

aug_train_gen = aug_train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=(256,256),
                                               interpolation='bilinear',
                                               color_mode='rgb',
                                               classes=None, 
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed)
                                               


val_data_gen = ImageDataGenerator(rescale=1/255.)

val_gen = val_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(256,256),
                                               interpolation='bilinear',
                                               color_mode='rgb',
                                               classes=None, 
                                               batch_size=32,
                                               shuffle=False,
                                               seed=seed)
                                              


In [ ]:
#Class weights due to unbalanced dataset
class_weights = class_weight.compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(aug_train_gen.classes),
                                        y = aug_train_gen.classes                                                    
                                    )
class_weights = dict(zip(np.unique(aug_train_gen.classes), class_weights))

print(class_weights)

In [ ]:
#Display some examples
image = next(aug_train_gen)[0][4]

t = aug_train_data_gen.get_random_transform(img_shape=input_shape, seed=seed)
print("Transform:", t)

augmented = aug_train_data_gen.apply_transform(image, t)

fig, ax = plt.subplots(1, 2, figsize=(15,30))
ax[0].imshow(image)
ax[0].set_title("Original")
ax[1].imshow(augmented)
ax[1].set_title("Augmented")
plt.show()

In [ ]:
def build_model(input_shape):
    #input
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    #Convolutional layers (conv+pool)*4
    conv1 = tfkl.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(input_layer)
    pool1 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv1)

    conv2 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool1)
    pool2 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv2)

    conv3 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool2)
    pool3 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv3)

    conv4 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool3)
    pool4 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv4)
    
    #top layers 
    flattening_layer = tfkl.Flatten(name='Flatten')(pool4)
    flattening_layer = tfkl.Dropout(0.3, seed=seed)(flattening_layer)
    classifier_layer = tfkl.Dense(units=512, name='Classifier', kernel_initializer=tfk.initializers.GlorotUniform(seed), activation='relu')(flattening_layer)
    classifier_layer = tfkl.Dropout(0.3, seed=seed)(classifier_layer)
    #output layer
    output_layer = tfkl.Dense(units=14, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='Output')(classifier_layer)

   # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')
  # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    return model

In [ ]:
from datetime import datetime
def create_callbacks():
  callbacks = []

  exps_dir = os.path.join('trained')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, 'ToyNotebook' + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'trained/checkpoint.ckpt')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp'),
                                                     save_weights_only=False,
                                                     save_best_only=True)
  callbacks.append(ckpt_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

In [ ]:
#create the model 
model = build_model(input_shape)

In [ ]:
tf.get_logger().setLevel('WARNING')

# Create folders and callbacks and fit
callbacks = create_callbacks()

history = model.fit(
    x = aug_train_gen,
    epochs = epochs,
    validation_data = val_gen,
    class_weight = class_weights,
    callbacks = callbacks
)

In [ ]:
#PLOTS FOR MODEL EVALUATION:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Compute the classification metrics
y_pred = np.argmax(model.predict_generator(val_gen, steps=len(val_gen)), axis=1) 
y_true = np.concatenate([np.argmax(val_gen[i][1], axis=1) for i in range(len(val_gen))])


val_accuracy = sum(y_pred == y_true) / len(y_true)
val_accuracy

In [ ]:
confusion_mtx = tf.math.confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx,
            xticklabels=labels,
            yticklabels=labels,
            annot=True, 
            cmap='Blues',
            fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()

In [ ]:
model.save("trained/modelloDaSubmittare")